In [1]:
import pandas as pd
import traitlets
import ipywidgets
import bqplot
import numpy as np

In [2]:
states = pd.read_csv("us-states.csv", parse_dates = ["date"])

In [3]:
states.loc

In [4]:
states.iloc

In [5]:
states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
states.loc[0:3]

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0


In [7]:
states.iloc[0:3]

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0


In [8]:
states_by_date = states.set_index("date")

In [9]:
states_by_date

,state,fips,cases,deaths
date,,,,
2020-01-21,Washington,53,1,0
2020-01-22,Washington,53,1,0
2020-01-23,Washington,53,1,0
2020-01-24,Illinois,17,1,0
2020-01-24,Washington,53,1,0
...,...,...,...,...
2020-10-13,Virginia,51,160805,3372
2020-10-13,Washington,53,99311,2311
2020-10-13,West Virginia,54,18555,387


In [10]:
states_by_date.loc['2020-01-21':'2020-01-23']

,state,fips,cases,deaths
date,,,,
2020-01-21,Washington,53,1,0
2020-01-22,Washington,53,1,0
2020-01-23,Washington,53,1,0


In [11]:
states_by_date.iloc[0:4]

,state,fips,cases,deaths
date,,,,
2020-01-21,Washington,53,1,0
2020-01-22,Washington,53,1,0
2020-01-23,Washington,53,1,0
2020-01-24,Illinois,17,1,0


In [12]:
states_by_date.loc['2020-01-21':'2020-01-25']

,state,fips,cases,deaths
date,,,,
2020-01-21,Washington,53,1,0
2020-01-22,Washington,53,1,0
2020-01-23,Washington,53,1,0
2020-01-24,Illinois,17,1,0
2020-01-24,Washington,53,1,0
2020-01-25,California,6,1,0
2020-01-25,Illinois,17,1,0
2020-01-25,Washington,53,1,0


In [13]:
states_by_date.groupby("state").max()["cases"]

state
Alabama                     167193
Alaska                       10960
Arizona                     226749
Arkansas                     94167
California                  863614
Colorado                     80414
Connecticut                  61697
Delaware                     22394
District of Columbia         16068
Florida                     738741
Georgia                     318065
Guam                          4235
Hawaii                       13736
Idaho                        49987
Illinois                    329160
Indiana                     140505
Iowa                        101531
Kansas                       69328
Kentucky                     83967
Louisiana                   175329
Maine                         5780
Maryland                    132839
Massachusetts               140412
Michigan                    152842
Minnesota                   114627
Mississippi                 105941
Missouri                    150496
Montana                      19682
Nebraska      

In [14]:
total_cases = states_by_date.groupby("date").sum()["cases"]

In [15]:
states_by_date.groupby("state").get_group("Illinois")

,state,fips,cases,deaths
date,,,,
2020-01-24,Illinois,17,1,0
2020-01-25,Illinois,17,1,0
2020-01-26,Illinois,17,1,0
2020-01-27,Illinois,17,1,0
2020-01-28,Illinois,17,1,0
...,...,...,...,...
2020-10-09,Illinois,17,317501,9207
2020-10-10,Illinois,17,320283,9235
2020-10-11,Illinois,17,322985,9245


In [16]:
states_timeseries = dict(tuple(_) for _ in states_by_date.groupby("state"))

In [17]:
states_timeseries['Illinois']

,state,fips,cases,deaths
date,,,,
2020-01-24,Illinois,17,1,0
2020-01-25,Illinois,17,1,0
2020-01-26,Illinois,17,1,0
2020-01-27,Illinois,17,1,0
2020-01-28,Illinois,17,1,0
...,...,...,...,...
2020-10-09,Illinois,17,317501,9207
2020-10-10,Illinois,17,320283,9235
2020-10-11,Illinois,17,322985,9245


In [18]:
case_counts = states.groupby("fips")["cases"].max().to_dict()

proj = bqplot.AlbersUSA()
color_sc = bqplot.ColorScale(scheme = 'Reds')
color_ax = bqplot.ColorAxis(scale = color_sc, label = "Case Count", reverse = True)

mark = bqplot.Map(map_data = bqplot.topo_load("map_data/USStatesMap.json"),
                  scales = {'projection': proj, 'color': color_sc},
                  color = case_counts,
                  colors = {17: '#ff0000'},
                  hovered_styles = {'hovered_fill': 'none',
                                    'hovered_stroke': 'black',
                                    'hovered_stroke_width': 5.0}
                 )
fig = bqplot.Figure(marks = [mark], axes = [color_ax])
display(fig)

Figure(axes=[ColorAxis(label='Case Count', scale=ColorScale(scheme='Reds'))], fig_margin={'top': 60, 'bottom':…

In [19]:
date_sc = bqplot.DateScale()
case_sc = bqplot.LogScale()

date_ax = bqplot.Axis(scale = date_sc)
case_ax = bqplot.Axis(scale = case_sc, orientation = 'vertical')

lines = bqplot.Lines(x = total_cases.index, y = total_cases,
                     scales = {'x': date_sc, 'y': case_sc})

interval_selector = bqplot.interacts.FastIntervalSelector(scale = date_sc)

fig = bqplot.Figure(marks = [lines], axes = [date_ax, case_ax], interaction = interval_selector)
display(fig)

Figure(axes=[Axis(scale=DateScale()), Axis(orientation='vertical', scale=LogScale())], fig_margin={'top': 60, …

In [20]:
case_counts = states_by_date.groupby("fips")["cases"].max().to_dict()

proj = bqplot.AlbersUSA()
color_sc = bqplot.ColorScale(scheme = 'Reds', min = states_by_date["cases"].min(), max = states_by_date["cases"].max())
color_ax = bqplot.ColorAxis(scale = color_sc, label = "Case Count", reverse = True)

mark = bqplot.Map(map_data = bqplot.topo_load("map_data/USStatesMap.json"),
                  scales = {'projection': proj, 'color': color_sc},
                  color = case_counts,
                  colors = {'default_color': 'white'},
                  hovered_styles = {'hovered_fill': 'none',
                                    'hovered_stroke': 'black',
                                    'hovered_stroke_width': 5.0}
                 )
fig_map = bqplot.Figure(marks = [mark], axes = [color_ax])

date_sc = bqplot.DateScale()
case_sc = bqplot.LogScale()

date_ax = bqplot.Axis(scale = date_sc)
case_ax = bqplot.Axis(scale = case_sc, orientation = 'vertical')

lines = bqplot.Lines(x = total_cases.index, y = total_cases,
                     scales = {'x': date_sc, 'y': case_sc})

interval_selector = bqplot.interacts.FastIntervalSelector(scale = date_sc)

fig_line = bqplot.Figure(marks = [lines], axes = [date_ax, case_ax], interaction = interval_selector)

def on_selection_change(change):
    if change['new'] is None: return
    start_date, stop_date = change['new']
    new_color = states_by_date.loc[start_date:stop_date].groupby("fips").max()["cases"].to_dict()
    mark.color = new_color

interval_selector.observe(on_selection_change, "selected")

display(ipywidgets.VBox([fig_map, fig_line]))

In [21]:
mark.interactions = {'click': 'select'}

In [22]:
mark.selected

In [23]:
case_counts = states_by_date.groupby("fips")["cases"].max().to_dict()

proj = bqplot.AlbersUSA()
color_sc = bqplot.ColorScale(scheme = 'Reds', min = states_by_date["cases"].min(), max = states_by_date["cases"].max())
color_ax = bqplot.ColorAxis(scale = color_sc, label = "Case Count", reverse = True)

mark = bqplot.Map(map_data = bqplot.topo_load("map_data/USStatesMap.json"),
                  scales = {'projection': proj, 'color': color_sc},
                  color = case_counts,
                  colors = {'default_color': 'white'},
                  hovered_styles = {'hovered_fill': 'none',
                                    'hovered_stroke': 'black',
                                    'hovered_stroke_width': 5.0}
                 )
mark.interactions = {'click': 'select'}
fig_map = bqplot.Figure(marks = [mark], axes = [color_ax])

date_sc = bqplot.DateScale()
case_sc = bqplot.LogScale()

date_ax = bqplot.Axis(scale = date_sc)
case_ax = bqplot.Axis(scale = case_sc, orientation = 'vertical')

lines = bqplot.Lines(x = total_cases.index, y = total_cases,
                     scales = {'x': date_sc, 'y': case_sc})

interval_selector = bqplot.interacts.FastIntervalSelector(scale = date_sc)

fig_line = bqplot.Figure(marks = [lines], axes = [date_ax, case_ax], interaction = interval_selector)

def on_state_selection_change(change):
    if change['new'] is None: return
    new_data = [total_cases]
    fips_groupby = states_by_date.groupby("fips")
    for fips_value in change['new']:
        new_data.append(fips_groupby.get_group(fips_value)["cases"])
    lines.y = pd.DataFrame(new_data)

mark.observe(on_state_selection_change, "selected")

def on_selection_change(change):
    if change['new'] is None: return
    start_date, stop_date = change['new']
    new_color = states_by_date.loc[start_date:stop_date].groupby("fips").max()["cases"].to_dict()
    mark.color = new_color

interval_selector.observe(on_selection_change, "selected")

display(ipywidgets.VBox([fig_map, fig_line]))